In [1]:
# [Cell 1] 기본 라이브러리 및 설정
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 설정값
INPUT_FILE = "project_full_context.txt"
OUTPUT_JSON = "project_flows.json"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 실행 환경: {device.upper()}")

c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 실행 환경: CUDA


In [ ]:
# [Cell 2] 모델 및 토크나이저 로딩
MODEL_ID = "microsoft/Phi-3.5-mini-instruct"  # 또는 사용 중인 모델 ID

try:
    if 'model' not in globals():
        print("🔄 모델 로딩 중...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True,
            attn_implementation="eager"
        )
        print("✅ 모델 로딩 완료!")
    else:
        print("✅ 모델이 이미 로드되어 있습니다.")
except Exception as e:
    print(f"❌ 모델 로딩 실패: {e}")

🔄 모델 로딩 중...


c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pak10\.cache\huggingface\hub\models--Qwen--Qwen2.5-Coder-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

✅ 모델 로딩 완료!


In [3]:
# [Cell 3] 프롬프트 매니저 (계층형 트리 구조 전용)

SYSTEM_PROMPT = """
You are a 'Backend Architecture Analyzer'.
Your goal is to generate a **Hierarchical Call Graph** in JSON format for the provided codebase.

### INSTRUCTIONS
1. **Identify Categories**: Group endpoints by feature (e.g., Auth, Post, User).
2. **Find Endpoints**: Look for Router definitions (e.g., `router.post('/signup', ...)`).
3. **Trace Recursively**: For each endpoint, trace the function calls deeper and deeper.
   - Router -> Middleware -> Controller -> Service/Data Layer -> Database/Library.
4. **Structure**: Use the `children` array to represent nested function calls.

### JSON OUTPUT FORMAT (STRICT)
You MUST follow this exact structure. Do not output markdown code blocks, just the JSON.

{
  "api": [
    {
      "category": "auth",
      "categoryName": "Auth Feature",
      "endpoints": [
        {
          "method": "POST",
          "url": "/auth/login",
          "function": "login",
          "file": "controller/auth.js",
          "description": "User Login",
          "children": [
            {
              "function": "validateLogin",
              "file": "middleware/valid.js",
              "description": "Validation Middleware",
              "children": [
                 { "function": "isEmail()", "file": "express-validator", "description": "Check email format", "children": [] }
              ]
            },
            {
              "function": "findByUserid",
              "file": "data/auth.js",
              "description": "Find user from DB",
              "children": [
                 { "function": "db.execute()", "file": "mysql2", "description": "SELECT * FROM users...", "children": [] }
              ]
            }
          ]
        }
      ]
    }
  ]
}
"""

print("✅ 계층형 분석 프롬프트 설정 완료")

✅ 계층형 분석 프롬프트 설정 완료


In [4]:
# [Cell 4] 전체 코드 분석 및 JSON 생성 엔진

def extract_json(text):
    """AI 응답에서 JSON 부분만 추출"""
    text = re.sub(r"^```(json)?", "", text.strip(), flags=re.MULTILINE)
    text = re.sub(r"```$", "", text.strip(), flags=re.MULTILINE)
    
    # 중괄호 균형 맞추기
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1: return "{}"
    return text[start:end+1]

def run_full_scan():
    if not os.path.exists(INPUT_FILE):
        print(f"❌ '{INPUT_FILE}' 파일이 없습니다.")
        return

    print(f"📂 '{INPUT_FILE}' 읽는 중...")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        full_code = f.read()

    # 코드가 너무 길면 앞부분 50,000자만 사용 (메모리 문제 방지)
    # Phi-3.5나 Qwen은 컨텍스트가 길어서 더 늘려도 됩니다.
    code_context = full_code[:60000] 

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analyze this entire codebase and generate the Nested JSON Structure:\n\n{code_context}"}
    ]

    print("🧠 AI가 전체 구조를 분석 중입니다... (시간이 조금 걸립니다)")
    
    # 추론
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    
    outputs = model.generate(
        input_ids,
        max_new_tokens=4000, # JSON이 길게 나오므로 토큰을 넉넉하게 줌
        temperature=0.1,
        do_sample=True
    )
    
    response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    
    # JSON 파싱 및 저장
    json_str = extract_json(response)
    
    try:
        data = json.loads(json_str)
        with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✅ 완벽한 JSON 파일이 생성되었습니다! -> {OUTPUT_JSON}")
        return data
    except json.JSONDecodeError as e:
        print("❌ JSON 파싱 실패. AI 응답을 확인하세요.")
        print(response[:500])
        return None

# 실행
json_data = run_full_scan()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


📂 'project_full_context.txt' 읽는 중...
🧠 AI가 전체 구조를 분석 중입니다... (시간이 조금 걸립니다)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# [Cell 5] 텍스트 트리 뷰어 (JSON 구조 확인용)

def print_tree(node, prefix="", is_last=True):
    """재귀적으로 JSON 트리를 출력"""
    connector = "└── " if is_last else "├── "
    
    # 노드 이름 결정 (function 또는 method+url)
    if "method" in node:
        name = f"[{node['method']}] {node['url']} ({node.get('description', '')})"
    elif "function" in node:
        name = f"ƒ {node['function']} - {node.get('description', '')}"
    elif "category" in node:
        name = f"📂 Category: {node.get('categoryName', node['category'])}"
    else:
        name = "Unknown Node"

    print(prefix + connector + name)
    
    # 자식 노드 순회
    children = node.get("children", [])
    if "endpoints" in node: # 카테고리 레벨
        children = node["endpoints"]
        
    count = len(children)
    for i, child in enumerate(children):
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_tree(child, new_prefix, i == count - 1)

def visualize_json_structure():
    if not os.path.exists(OUTPUT_JSON):
        print("❌ JSON 파일이 없습니다.")
        return

    with open(OUTPUT_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print("\n🌳 API Call Graph Analysis Result\n" + "="*40)
    if "api" in data:
        for cat in data["api"]:
            print_tree(cat)
    else:
        print("⚠️ 'api' 키를 찾을 수 없습니다. JSON 구조를 확인하세요.")

# 실행
visualize_json_structure()

🔍 'createJwtToken' 발견! (코드 위치: 667 ~ 4667 추출)
🧠 AI 분석 시작: 'createJwtToken' (모드: deep_dive)
✅ JSON 생성 성공! (project_flows.json)


True

In [ ]:
import sys
import transformers
import accelerate
import torch

print(f"📂 파이썬 실행 위치: {sys.executable}")
print("-" * 30)
print(f"❌ Transformers 버전: {transformers.__version__}")
print(f"❌ Accelerate 버전  : {accelerate.__version__}")
print(f"❌ PyTorch 버전     : {torch.__version__}")
print("-" * 30)
print("※ Transformers 버전이 '4.38.0' 미만이면 CMD에서 아무리 설치해도 여기선 적용 안 된 겁니다.")

📂 파이썬 실행 위치: c:\Pyg\Projects\semi\yuzyproject-aimodels\venv\Scripts\python.exe
------------------------------
❌ Transformers 버전: 4.57.3
❌ Accelerate 버전  : 1.12.0
❌ PyTorch 버전     : 2.9.1+cpu
------------------------------
※ Transformers 버전이 '4.38.0' 미만이면 CMD에서 아무리 설치해도 여기선 적용 안 된 겁니다.
